# `menu.csv` __Data Cleaning__

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
df = pd.read_csv("./Menu.csv")

## __Step 1__

- Missing Value Correction
- Date Standardization, Date Outlier

### __Rationale:__

One might assume that missing values can be filled with derived data from foreign keys.

For example, if a date is missing for row X in the `Menu.csv` table but a `created_at`
or `updated_at` value exists for X.name -> menu_id -> menu_page_id, then the original
missing date can transitively become the existing `created_at` or `updated_at`. The same
example applies to the other two columns, `currency` and `location`.

The problem with the example above is that it presupposes the authenticity of the data
across multiple tables, which is dangerous without proper data validation methods.
Although a missing entry can have a derivable value based on data from a corresponding
data point tied to the missing entry's primary key, additional validation must occur
because the instance of missing data negatively impacts the trust in the data between
multiple tables for the same flawed primary key.

By inspection, there is a drastic reduction in rows after removing missing values;
however, deriving data across the tables cannot be validated for the scope of this
analysis. Regardless, there is ample data remaining to accomplish the target use case
set out in the initial proposal for an algorithmic pricing model based on price and
location over time.

In [ ]:
df_cleaned = df

# Replace empty strings with NaN
df_cleaned.replace("", pd.NA, inplace=True)

# Remove the rows with missing values in the "date", "currency", and "location" columns
df_cleaned = df_cleaned.dropna(subset=["date"])
df_cleaned = df_cleaned.dropna(subset=["currency"])
df_cleaned = df_cleaned.dropna(subset=["location"])
display(df_cleaned)

In [ ]:
# Standardize the entire date column to the ISO format

# Helper function to check and convert a date to ISO format
def to_iso_format(date_str):
    try:
        # Try to parse the date in various common formats
        date = pd.to_datetime(date_str, errors="raise")

        # Return the date in ISO format
        return date.strftime("%Y-%m-%d")
    except Exception as e:
        print(f"Error parsing date: {date_str} -> {e}")
        return None

# Apply the date function to the date column
df_cleaned["date"] = df_cleaned["date"].apply(to_iso_format)

# Drop rows where the date could not be parsed
df_cleaned = df_cleaned.dropna(subset=["date"])

display(df_cleaned)

In [ ]:
# Inspect dates further to verify if more outliers exist

# Ensure the date column is in datetime format
df_cleaned_sorted_by_dates = df_cleaned
df_cleaned_sorted_by_dates["date"] = pd.to_datetime(df["date"], errors="coerce")

# Sort the DataFrame by the date column
df_cleaned_sorted_by_dates = df_cleaned_sorted_by_dates.sort_values(by="date")

__Note:__ the dates range from 1851 to 2015 which is reasonable for this dataset, meaning no further cleaning is necessary for the dates

In [ ]:
# Display the earliest dates
display(df_cleaned_sorted_by_dates.head(10))

# Display the latest dates
display(df_cleaned_sorted_by_dates.tail(10))

In [ ]:
# Save the intermediately cleaned dataset
df_cleaned.to_csv("./CleanedMenuStep1.csv", index=False)

## __Step 2__

- Currency Standardization, Price Outlier 


In [ ]:
df = pd.read_csv("./CleanedMenuStep1.csv")

In [ ]:
df = df[~((df['currency'].isna() | (df['currency'].str.strip() == '')) & 
          (df['currency_symbol'].isna() | (df['currency_symbol'].str.strip() == '')))]

In [ ]:
unique_currencies = df['currency'].unique()

In [ ]:
df = df[~df['currency'].isin(['Cents', 'Pence'])]
unique_currencies_2 = df['currency'].unique()

In [ ]:
unique_combinations = df[['currency', 'currency_symbol']].drop_duplicates()

In [ ]:
currency_to_symbol = {
    'Dollars': 'USD',  # Generally USD for US Dollars, but can vary (e.g., CAD for Canadian Dollars)
    'Francs': 'FRF',  # ISO code for French Francs; Belgian Francs also used FRF, but ₣ is the historical symbol
    'Belgian Francs': 'BEF',
    'Shillings': 'SHP',  # ISO code for Saint Helena Pound (historically shillings)
    'Deutsche Marks': 'DEM',
    'UK Pounds': 'GBP',
    'Canadian Dollars': 'CAD',
    'Austro-Hungarian Kronen': 'HUF',  # ISO code for Hungarian Forint; Kronen does not have a modern ISO code
    'Swiss Francs': 'CHF',
    'Pesetas': 'ESP',  # ISO code for Spanish Peseta
    'Danish kroner': 'DKK',
    'Swedish kronor (SEK/kr)': 'SEK',
    'Yen': 'JPY',
    'Italian Lire': 'ITL',
    'Quetzales': 'GTQ',
    'Israeli lirot (1948-1980)': 'ILS',
    'Dutch Guilders': 'NLG',  # ISO code for Dutch Guilder
    'Austrian Schillings': 'ATS',  # ISO code for Austrian Schilling
    'Escudos': 'PTE',  # ISO code for Portuguese Escudo
    'Euros': 'EUR',
    'Bermudian dollars': 'BMD',
    'Hungarian forint': 'HUF',
    'Mexican pesos': 'MXN',
    'Drachmas': 'GRD',
    'New Taiwan Dollar': 'TWD',
    'Icelandic Krónur': 'ISK',
    'Australian Dollars': 'AUD',
    'Argentine peso': 'ARS',
    'Sol': 'PEN',
    'Uruguayan pesos': 'UYU',
    'Brazilian Cruzeiros': 'BRB',  # ISO code for Brazilian Cruzeiro
    'Złoty': 'PLN',
    'Norwegian kroner': 'NOK',
    'Cuban pesos': 'CUP',
    'Finnish markka': 'FIM',
    'Lats': 'LVL',  # ISO code for Latvian Lats
    'Straits dollar (1904-1939)': 'SGD'  # ISO code for Singapore Dollar (used as reference for historical dollar)
}

In [ ]:
# Replace currency_symbol based on currency using the mapping dictionary
df['currency_symbol'] = df['currency'].map(currency_to_symbol).fillna(df['currency_symbol'])

In [ ]:
unique_combinations_after_update = df[['currency', 'currency_symbol']].drop_duplicates()

In [ ]:
df.to_csv('CleanedMenuStep2.csv', index=False)

## __Step 3__

- Event Standardization, Event Outlier


In [ ]:
df = pd.read_csv("CleanedMenuStep2.csv")

In [ ]:
# Preprocess the data: remove problematic characters and strip whitespace
df["occasion_cleaned"] = df["occasion"].str.replace(r'[^\w\s]', '', regex=True).str.strip().str.upper().fillna("")

In [ ]:
# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Generate embeddings for the occasions
print("Generating embeddings...")
embeddings = model.encode(df["occasion_cleaned"].tolist(), show_progress_bar=True)

In [ ]:
# Determine the number of clusters (categories) - arbitrarily defined but could do spaghetti analysis in the future
num_clusters = 20

In [ ]:
# Apply KMeans clustering
print("Clustering embeddings...")
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(embeddings)

In [ ]:
# Add cluster labels to the DataFrame
df["category"] = kmeans.labels_

In [ ]:
# Print the categories to manually label them
for i in range(num_clusters):
    print(f"Cluster {i}:")
    print(df[df["category"] == i]["occasion_cleaned"].tolist())
    print("\n")

In [ ]:
# Manually determined mapping from cluster labels to category names
cluster_to_category = {
    0: "Anniversary",
    1: "Daily",
    2: "Complimentary",
    3: "Annual",
    4: "Farewell",
    5: "Tour",
    6: "Holiday",
    7: "Patriotic",
    8: "Rite",
    9: "Dinner",
    10: "Breakfast",
    11: "Social",
    12: "Meeting",
    13: "Religious Holiday",
    14: "Political",
    15: "Festival",
    16: "Reunion",
    17: "Reception",
    18: "Lunch",
    19: "Graduation",
}

In [ ]:
# Map cluster labels to category names
df["category_name"] = df["category"].map(cluster_to_category)

In [5]:
# Remove empties
df.loc[df["occasion_cleaned"] == "", "category_name"] = ""

In [ ]:
# Save the categorized DataFrame
df.to_csv("CleanedMenuStep3.csv", index=False)